In [2]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import pickle
import numpy as np

import nltk
import gensim
from unidecode import unidecode

# Main working parts

First, need to scrape for the data. In this case, I scrape only for American restaurants in San Francisco, and later hope to get more data. 

Cory taught me to use the headers, otherwise sites seem to block me from scraping their data.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = 'http://sanfrancisco.menupages.com/restaurants/all-areas/all-neighborhoods/american/'
page = requests.get(url,headers=headers)


I need to scrape for restaurant menus. The first thing I need to do is obtain unique identifiers to append to the menu url to get the data for each restaurant.

After some exploration, I found that the url for each restaurant can be found between the strings "restaurants/" and "/" in the html data. I then managed to use re to extract all the data, and store it in a variable called rests.

In [ ]:
rests = re.findall(r'restaurants/(.+?)/',page.content)

I then found that there is an entry called advertise in there. I'm not sure how many there are, so I remove anything that looks like advertise and store it in a variable called amrests. I verified that I have 630 results for American restaurants on menupages, and exactly 630 entries in my list.

In [ ]:
regex = re.compile(r"advertise")
amrests = [i for i in rests if not regex.search(i)]

I then want to obtain the restaurant names. I find that the names lie between the strings " "\'> " and " < / a > " (without the quotations and the spaces). I extract them and store them in a list called names. Once I remove all the empty strings, I verify that there are 630 names in this list as well.

Sanity check later: Ensure all names are unique

In [ ]:
names = re.findall(r'"\'>(.*?)</a>',page.content)
names = [i for i in names if i]

I then wanted to pull the addresses of the restaurants. I found (somewhat) that these addresses lay between " < br / > " and the pipe symbol " | ". To reference the pipe symbol in re, I had to use a backslash, which took a while to figure out and gives a hint for gotchas in the future.

*I only got 619 addresses, so there is some inconsistency. For now, I think it's ok because for MVP I'm not looking to find the nearest restaurant, just a restaurant. Very important to come back to this later.*

In [ ]:
adds = re.findall(r'<br/> (.+?) \|',page.content)

Using the menu urls, I scrape the menupages websites and extract the menus. For american restaurants, there are 630 menus and this takes roughly 10 minutes to get through. I store the resuls in a hashtable (dictionary)

In [ ]:
menu_dict = {}
t0 = time.time()

for i in amrests:
    url = 'http://sanfrancisco.menupages.com/restaurants/' + i + '/menu'
    menu_dict[i] = requests.get(url,headers=headers).content
t1 = time.time()
totalt = t1-t0
print totalt

As a sanity check, I ensure that there are 630 menus

In [ ]:
len(menu_dict)

I store the messy menu html files in a pandas dataframe

In [ ]:
amrest_df = pd.DataFrame(menu_dict.items())

With a view towards more categories, I categorize all of these items as American food. However, this will probably not be useful for MVP.

In [ ]:
amrest_df.loc[:,'Type'] = 'American'

I needed to go home, so I pickled the dataframe, and also the page html content, restuarant urls, names and addresses. On Monday, I modified the dataframe, added names etc

In [ ]:
#zip(amrests,names)
#amrest_df = amrest_df.rename(columns={0:'URL',1:'HTML_Menu'})
#amrest_df['Names'] = amrest_df['URL']
#amrest_df.head()
# name_url_dict = {}
# for i,j in enumerate(amrests):
#     name_url_dict[j] = names[i]
#amrest_df['Names'] = amrest_df['Names'].map(name_url_dict)
amrest_df.tail()

In [ ]:
amrest_df.to_pickle('amrest_df')

In [ ]:
# obj0, obj1, obj2 are created here...
# page_content = page.content
# Saving the objects:
# with open('objs.pickle', 'w') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([page_content, rests, names, adds], f)

# Getting back the objects:
with open('objs.pickle') as f:  # Python 3: open(..., 'rb')
    page_content, rests, names, adds = pickle.load(f)

*Noise in present manner of data extraction (need to address later)*
    
1) Not taking into account the context (sometimes the price is listed as bacon for \$9.49 but really it's meant to be a 2 egg breakfast for \$9.49. This is pertinent in It's Top's Coffee Shop
menu. Need a better way of handling this in the data
    
2) Some prices are in title. When price is non existent, need to pull from title (See It's Top's Coffee Shop House omlettes \$7.95)

3) Some items have weird pricing (See It's Top's Coffee Shop Side of eggs)

4) Need to add context (Side of eggs vs breakfast consisting of eggs). This can be obtained from headers.

5) Can extract more ingredients from description, but this will require manipulation of code to extract headers (see above) and text below headers

##### Extracting menu item, description and price from the garbled HTML data

In [ ]:
menu_df_dict ={}

regexp_menu_item = r'<cite>(.+?)</cite>'
el_menu_item = 0
regexp_item_description = r'\xa0(.+?)</th>'
el_item_description = 0
regexp_price = r'\xa0(.+?)\r'
el_price = 1

menu_index_start = 2

for i in range(amrest_df.shape[0]):
    menu_html = amrest_df['HTML_Menu'][i]
    bs_menu = BeautifulSoup(menu_html,'html.parser')
    menu_items_list = bs_menu.find_all('tr')
    
    menu_item_list = createItemList(menu_items_list[menu_index_start:],regexp_menu_item,el_menu_item)
    item_description_list = createItemList(menu_items_list[menu_index_start:],regexp_item_description,el_item_description)
    price_list = createItemList(menu_items_list[menu_index_start:],regexp_price,el_price)
    
    menu_df_dict[amrest_df['URL'][i]] = pd.DataFrame(zip(menu_item_list,item_description_list,price_list),columns=['Menu Item','Item Description','Price'])
    
def createItemList(bsoup_list,regexp,element):
    new_list = []
    for y in [re.findall(regexp,str(x)) for x in bsoup_list]:
        try:
            new_list.append(y[element])
        except:
            new_list.append("")    
    
    return new_list

###### Pickle the data

In [ ]:
amrest_df = pd.read_pickle('amrest_df')

# pickle.dump(menu_df_dict, open( "menu_df_dict.p", "wb" ) )

menu_df_dict = pickle.load( open( "menu_df_dict.p", "rb" ) )

###### Append the actual restaurant name, and the concatenated menu and item description strings to each menu dataframe that is associated with an individual restaurant

In [ ]:
for i,j in zip(amrest_df['URL'],amrest_df['Names']):
    #menu_df_dict[i]['Menu Item + Description Text'] = menu_df_dict[i]['Menu Item'].map(str) + " " + menu_df_dict[i]['Item Description'].map(str)
    menu_df_dict[i]['Restaurant Name'] = j

###### Concatenate the entire dictionary of dataframes together in one "giant" dataframe. 

In [ ]:
all_amrests_df = pd.concat(menu_df_dict.values(), ignore_index=True)

# Save the dateframe. For this week, start from here

In [ ]:
#all_amrests_df.to_pickle('all_amrests_df')
all_amrests_df = pd.read_pickle('all_amrests_df')

In [ ]:
import string
# Remove punctuations
all_amrests_df['desc_list'] = all_amrests_df['Menu Item + Description Text'].apply(lambda x: x.translate(string.maketrans("",""), string.punctuation))
# Tokenize
all_amrests_df['desc_list'] = all_amrests_df['desc_list'].apply(lambda x: x.split())

In [ ]:
# Remove stopwords, make everything lowercase
all_amrests_df['desc_list'] = all_amrests_df['desc_list'].apply(lambda x: [i.lower() for i in x if i.lower() not in nltk.corpus.stopwords.words('english')]) 

In [ ]:
# Get a list of list of tokens
tokens_list = all_amrests_df.desc_list.tolist()

# Get WordNet Lemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

# Lemmatize the tokens, so run, runs and running are all mapped to run
all_amrests_df['desc_list_lem'] = [[lmtzr.lemmatize(i.decode('utf-8')) for i in x] for x in tokens_list]

In [ ]:
all_amrests_df.head(3)

In [ ]:
# Extract the lemmatized tokens
tokens = all_amrests_df['desc_list_lem'].tolist()

In [ ]:
# Vectorize the tokens using tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False)
tfs = tfidf.fit_transform(tokens)


In [ ]:
tfs
with open('tfidfmat.pickle', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(tfs, f)

In [ ]:
all_amrests_df['tfidf_vector'] = [i for i in tfs]

In [ ]:
all_amrests_df.iloc[600].tfidf_vector

#### Get the cosine similarity between an input documents and the rest of the documents, pull out the top 5 that look the same

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
# 500 works well
# 300 also works very well
#250
# 40000 is interesting
# 30000 works
# 50 is an example of working poorly

cosine_similarities = cosine_similarity(tfs[2500], tfs).flatten()
related_food_idcs = cosine_similarities.argsort()[::-1][1:6]

#cosine_similarities[related_food_idcs]

print related_food_idcs

all_amrests_df.iloc[related_food_idcs][["Menu Item","Item Description","Restaurant Name"]]

In [ ]:
pd_rest_table.iloc[18724]['Menu Item']

In [ ]:
cosine_similarities.argsort()[::-1][1:6]

In [ ]:
all_amrests_df.iloc[299]['Item Description']

##### Store results in a SQL database

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [ ]:
dbname = 'rest_db'
username = 'harisk87'
pswd = '2PsWrD!'

In [ ]:
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print engine.url

In [ ]:
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))


In [ ]:
#pd_rest_table = pd_rest_table.rename(columns={'MenuItem': 'Menu Item', 'ItemDescription': 'Item Description','RestaurantName' : 'Restaurant Name'})
#pd_rest_table = pd_rest_table.replace({'Restaurant Name': {'It&#39;s Top&#39;s Coffee Shop': 'It\'s Tops Coffee Shop'}})
pd_rest_table.head()

In [ ]:
pd_rest_table['Restaurant Name'] = [x.decode('utf-8') for x in pd_rest_table['Restaurant Name']]
pd_rest_table['Item Description'] = [x.decode('utf-8') for x in pd_rest_table['Item Description']]
pd_rest_table['Menu Item'] = [x.decode('utf-8') for x in pd_rest_table['Menu Item']]

In [ ]:
#pd_rest_table.to_pickle('pd_rest_table')
pd_rest_table = pd.read_pickle('pd_rest_table')
## pd_rest_table contains decoded version of stuff
#pd_rest_table = pd.read_pickle('pd_rest_table')
pd_rest_table.head()

In [ ]:
#pd_rest_table['tfidf_vector'].apply(lambda x: x.todense())
#too big!
pd_rest_table = pd_rest_table[["Menu Item","Item Description","Restaurant Name","Price"]]

In [ ]:
pd_rest_table.to_sql('rest_table', engine, if_exists='replace')

In [ ]:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [ ]:
sql_query = """
SELECT * FROM rest_table;
"""
rest_data_from_sql = pd.read_sql_query(sql_query,con)

In [ ]:
sql_query = """ SELECT DISTINCT "Restaurant Name" FROM rest_table """
test = pd.read_sql_query(sql_query,con)

In [ ]:
test.iloc[1]['Restaurant Name']

In [ ]:
rests = []
#query_results = rest_data_from_sql
query_results = pd_rest_table

In [ ]:
rests = []
for i in range(0,query_results.shape[0]):
    try:
        rests.append(dict(index=query_results.iloc[i]['index'], menu_item=query_results.iloc[i]['Menu Item'], desc=query_results.iloc[i]['Item Description'], name = query_results.iloc[i]['Restaurant Name'], price = query_results.iloc[i]['Price']) )
    except:
        #rests.append(dict(index='Weird codec error', menu_item='Weird codec error', desc='Weird codec error', name = 'Weird codec error', price = 'Weird codec error') )
        print i
        rests = []
        break

In [ ]:
len(rests)

In [ ]:
def encoding_hack(text):
    text_int = text.encode("utf-8")
    out_text = text_int.decode("utf-8").encode('ascii','ignore')
    return out_text

In [ ]:
badidx = []
for i in range(0,query_results.shape[0]):
    index_print = query_results.iloc[i]['index']
    
    try:
        menu_print = query_results.iloc[i]['Menu Item']
        desc_print = query_results.iloc[i]['Item Description']
        name_print = query_results.iloc[i]['Restaurant Name']
        price_print = query_results.iloc[i]['Price']
        
        menu_print = encoding_hack(menu_print)
        desc_print = encoding_hack(desc_print)
        name_print = encoding_hack(name_print)
        price_print = encoding_hack(price_print)
        
    except:
        badidx.append(i)

    

    #rests.append(dict(index=index_print, menu_item=menu_print, desc=desc_print, name=name_print, price=price_print) )


In [ ]:
with open('badidx.pickle', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump(badidx, f)

In [ ]:
with open('badidx.pickle') as f:  # Python 3: open(..., 'rb')
    badidx = pickle.load(f)

In [ ]:
pd_rest_table_dropped = query_results[~query_results.index.isin(badidx)]

In [ ]:
pd_rest_table_dropped.to_sql('rest_table', engine, if_exists='replace',index='False')

In [ ]:
pd_rest_table_dropped.loc[903]['Menu Item']

In [ ]:
pd_rest_table.head()

In [ ]:
all_amrests_df[(all_amrests_df['Restaurant Name'] == 'Delancey Street') & (all_amrests_df['Menu Item'] == 'Filet Of Smoked Trout')].index.values[0]

In [ ]:
all_amrests_df.shape

In [ ]:
pd_rest_table.shape

In [ ]:
pd_rest_table[(pd_rest_table['Restaurant Name'] == 'Delancey Street') & (pd_rest_table['Menu Item'] == 'Filet Of Smoked Trout')]

In [ ]:
pd_rest_table.iloc[895]

# PART 2: ATTEMPT TO GET CLEANER RESULTS

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = 'http://www.allmenus.com/ca/san-francisco/420269-thermidor/menu/'
page = requests.get(url,headers=headers)




In [45]:
soup= BeautifulSoup(page.text, "lxml")




In [49]:

url = 'http://www.allmenus.com/ca/san-francisco/-/american/'
page = requests.get(url,headers=headers)


In [100]:
food_categories = ['american','american-new','german','crepes','french','burgers','deli']#'asianfusion','californian','chinese','dim-sum','sandwiches'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

base_url = url = 'http://www.allmenus.com/ca/san-francisco/'

rest_url_list = []
category_list = []

print 'Scraping for urls'
for fcat in food_categories:
    # Get all the restaurants that food category falls into
    top_page = requests.get(base_url + '-/' + fcat + '/',headers=headers)
    
    # Turn it into a BeautifulSoup object
    top_soup= BeautifulSoup(top_page.text, "lxml")
    all_rest_links = top_soup.findAll("p",{"class","restaurant_name"})
    
    for rest in all_rest_links:
        rest_url_list.append(rest.find('a')['href'])
        category_list.append(fcat)
    
print 'URLs obtained, time to scrape for menus'

Scraping for urls
URLs obtained, time to scrape for menus


In [101]:

t0 = time.time()

item_list = []
i=1
for url,fcat in zip(rest_url_list,category_list):
    #print "Scraping restaurant %d"%(i)
    
    page = requests.get('http://www.allmenus.com'+url,headers=headers)
    soup = BeautifulSoup(page.text, "lxml")
    
    # Extract restaurant name
    name = unidecode(soup.find("h1", {"itemprop":"name"}).text)

    # Extract street address
    saddr = unidecode(soup.find("span",{"itemprop":"streetAddress"}).text)

    # Extract the city
    city = unidecode(soup.find("span",{"itemprop":"addressLocality"}).text)

    # Extract the state
    state = unidecode(soup.find("span",{"itemprop":"addressRegion"}).text)

    # Extract the zip code
    zipc = unidecode(soup.find("span",{"itemprop":"postalCode"}).text)
    
    # Extract yelp rating
    try:
        yelp_rating = float(soup.find("meta",{"itemprop":"ratingValue"})['content'])
    except:
        yelp_rating = None

    # Extract number of yelp reviews
    try:
        num_yelp_reviews = int(soup.find("meta",{"itemprop":"reviewCount"})['content'])
    except:
        num_yelp_reviews = None

    # Get the yelp link
    try:
        yelp_link = soup.find("span",{"class":"review_count"}).find('a')['href']
    except:
        yelp_link = None

    all_categories = soup.find_all("div",{"class":"category"})

    for cat in all_categories:
        category_name = unidecode(cat.find("div",{"class":"category_head"}).h3.text)
        category_description = unidecode(cat.find("div",{"class":"category_head"}).p.text)
    
        all_menu_items_in_category = cat.find_all("li",{"class":"menu_item"})
    
        for menu_item in all_menu_items_in_category:
            item_name = unidecode(menu_item.find("span",{"class":"name"}).text)
            item_description = unidecode(menu_item.find("p",{"class":"description"}).text)
            try:
                item_price = unidecode(menu_item.find("span",{"class":"price"}).text)
            except:
                item_price = []
        
            new_item = {'restaurant_name':name, 'item_name':item_name,'item_description':item_description,'item_price':item_price,'category_name'
                    :category_name,'category_description':category_description,'street_address':saddr,'city':city,
                    'state':state,'zip':zipc,'full_address':", ".join([saddr,city,state,zipc]),"yelp_rating":yelp_rating,
                   'num_reviews':num_yelp_reviews,'yelp_link':yelp_link,'restaurant_category':fcat}
            item_list.append(new_item)
    i+=1

test_df = pd.DataFrame(item_list)
del item_list
t1 = time.time()
print str(t1-t0) + 'seconds'

1076.99184704seconds


In [103]:
all_menus_rest_df = test_df

In [113]:
all_menus_rest_df.to_pickle('all_menus_rest_df.p')

TypeError: coercing to Unicode: need string or buffer, list found